# test roll out predictions with GP

In [25]:
import numpy as np
import matplotlib.pyplot as plt
import os
from link_bot_gaussian_process import link_bot_gp, data_reformatting
from link_bot_notebooks import toy_problem_optimization_common as tpoc
import tensorflow as tf
from IPython.display import HTML
import warnings
import gpflow as gpf
import h5py

tf.logging.set_verbosity(tf.logging.ERROR)
np.set_printoptions(suppress=True, precision=4, linewidth=200)

In [27]:
sdf, _, _, _ = tpoc.load_sdf("/home/pmitrano/catkin_ws/src/link_bot/link_bot_teleop/data/obstacles_1_sdf.npz")

In [5]:
gp_model_dir = '/home/pmitrano/catkin_ws/src/link_bot/link_bot_gaussian_process/log_data/separate_independent/May_31_18-01-04__bf505bde20__gpf'

gpf.reset_default_graph_and_session()
config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=False, per_process_gpu_memory_fraction=0.01))
gpf.reset_default_session(config=config)

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=h5py.h5py_warnings.H5pyDeprecationWarning)
    fwd_gp_model = link_bot_gp.LinkBotGP()
    fwd_gp_model.load(os.path.join(gp_model_dir, 'fwd_model'))
    
    inv_gp_model = link_bot_gp.LinkBotGP()
    inv_gp_model.load(os.path.join(gp_model_dir, 'inv_model'))

Loading model from /home/pmitrano/catkin_ws/src/link_bot/link_bot_gaussian_process/log_data/separate_independent/May_31_18-01-04__bf505bde20__gpf/fwd_model
Loading model from /home/pmitrano/catkin_ws/src/link_bot/link_bot_gaussian_process/log_data/separate_independent/May_31_18-01-04__bf505bde20__gpf/inv_model


In [ ]:
initial_config = np.array([[-2, 0, -1, 0, 0, 0]])
steps = 500
controls =  np.tile([1,0], [steps, 1])
durations = np.ones(steps, dtype=np.int)
prediction = link_bot_gp.predict(fwd_gp_model, initial_config, controls, durations)
anim = link_bot_gp.animate_predict(prediction, sdf, arena_size=20)
plt.close()
HTML(anim.to_jshtml())